In [88]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [92]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "BK app") 

sc = SparkContext(conf=conf)

In [93]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2020-09-05 20:38 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2020-09-05 20:38 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2020-09-05 20:38 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2020-09-05 20:38 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2020-09-05 20:38 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2020-09-05 20:38 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2020-09-05 20:38 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2020-09-05 20:38 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2020-09-05 20:38 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2020-09-05 20:38 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2020-09-05 20:38 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2020-09-05 20:38 /labs/laba01/ml-100k/u2.base


In [94]:
data = sc.textFile("/labs/laba01/ml-100k/u.data")

In [95]:
data.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

From Readme

In [27]:
# 'DETAILED DESCRIPTIONS OF DATA FILES',
#  '==============================================',
#  'u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.',
#  '              Each user has rated at least 20 movies.  Users and items are',
#  '              numbered consecutively from 1.  The data is randomly',
#  '              ordered. This is a tab separated list of ',
#  '\t         user id | item id | rating | timestamp. ',

In [96]:
df_all = data.map(lambda x: x.split("\t"))

In [97]:
df_all.take(10)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596'],
 ['298', '474', '4', '884182806'],
 ['115', '265', '2', '881171488'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817'],
 ['6', '86', '3', '883603013']]

In [98]:
df_filtered = df_all.filter(lambda x: x[1] == '22')

In [99]:
df_filtered.take(10)

[['269', '22', '1', '891448072'],
 ['8', '22', '5', '879362183'],
 ['90', '22', '4', '891384357'],
 ['222', '22', '5', '878183285'],
 ['313', '22', '3', '891014870'],
 ['145', '22', '5', '875273021'],
 ['326', '22', '4', '879874989'],
 ['217', '22', '5', '889069741'],
 ['271', '22', '5', '885848518'],
 ['7', '22', '5', '891351121']]

In [200]:
dict_all = data.map(lambda x: (x.split("\t")[2], 1)) \
                .sortByKey() \
                .countByKey()

In [201]:
dict_all

defaultdict(int, {'1': 6110, '2': 11370, '3': 27145, '4': 34174, '5': 21201})

In [202]:
list_all = list(dict_all.values())

In [203]:
list_all

[6110, 11370, 27145, 34174, 21201]

In [205]:
dict_film = data.map(lambda x: (x.split("\t"))) \
                .filter(lambda x: x[1] == '22') \
                .map(lambda x: (x[2], 1)) \
                .sortByKey() \
                .countByKey()

In [206]:
dict_film

defaultdict(int, {'1': 5, '2': 14, '3': 46, '4': 98, '5': 134})

In [207]:
list_film = list(dict_film.values())

In [208]:
list_film

[5, 14, 46, 98, 134]

In [212]:
import json
answer = {}
answer["hist_all"] = list_all
answer["hist_film"] = list_film
answer

{'hist_all': [6110, 11370, 27145, 34174, 21201],
 'hist_film': [5, 14, 46, 98, 134]}

In [213]:
with open('lab01.json', 'w') as outfile:
    json.dump(answer, outfile)

In [214]:
sc.stop()